In [1]:
# import required libraries
import os
import os.path
import numpy as np
import pandas as pd
from math import sqrt
from heapq import nlargest
from tqdm import trange
from tqdm import tqdm
from scipy import stats
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error

In [2]:
# Load the dataset
path = 'ml-100k'
def getData(folder_path, file_name):
    fields = ['userID', 'itemID', 'rating', 'timestamp']
    data = pd.read_csv(os.path.join(folder_path, file_name), sep='\t', names=fields)
    return data 

rating_df = getData(path, 'u.data')

In [3]:
rating_df.head()

,userID,itemID,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
num_users = len(rating_df.userID.unique())
num_items = len(rating_df.itemID.unique())
print("Number of users:", num_users)
print("Number of items:", num_items)

Number of users: 943
Number of items: 1682


### 1A

In [5]:
def dataPreprocessor(rating_df, num_users, num_items):
    """
        INPUT: 
            data: pandas DataFrame. columns=['userID', 'itemID', 'rating' ...]
            num_row: int. number of users
            num_col: int. number of items
            
        OUTPUT:
            matrix: 2D numpy array. 
            
        NOTE 1: see where something very similar is done in the lab in function 'buildUserItemMatrix'    
            
        NOTE 2: data can have more columns, but your function should ignore 
              additional columns.
    """
    ########### your code goes here ###########

    # Initialize a of size (numUsers, numItems) to zeros
    matrix = np.zeros((num_users, num_items), dtype=np.int8)
    
    # Populate the matrix based on the dataset
    for (index, userID, itemID, rating, timestamp) in rating_df.itertuples():
        matrix[userID-1, itemID-1] = rating
    
    
    ###########         end         ###########
    return matrix

In [6]:
dataPreprocessor(rating_df, num_users, num_items)

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]], dtype=int8)

### 1B

In [7]:
class BaseLineRecSys(object):
    def __init__(self, method, processor=dataPreprocessor):
        """
            method: string. From ['popularity','useraverage']
            processor: function name. dataPreprocessor by default
        """
        self.method_name = method
        self.method = self._getMethod(self.method_name)
        self.processor = processor
        self.pred_column_name = self.method_name
        
    def _getMethod(self, method_name):
        """
            Don't change this
        """
        switcher = {
            'popularity': self.popularity,
            'useraverage': self.useraverage,
        }
        
        return switcher[method_name]
    
    @staticmethod
    def useraverage(train_matrix, num_users, num_items):
        """
            INPUT:
                train_matrix: 2D numpy array.
                num_users: int. Number of Users.
                num_items: int. Number of Items.
            OUTPUT:
                predictionMatrix: 2D numpy array.
                
            NOTE: see where something very similar is done in the lab in function 'predictByUserAverage'    
        """       
        predictionMatrix = np.zeros((num_users, num_items))
        ########### your code goes here ###########
        # Initialize the predicted rating matrix with zeros
            # Initialize the predicted rating matrix with zeros

        for (user,item), rating in np.ndenumerate(train_matrix):
            # Predict rating for every item that wasn't ranked by the user (rating == 0)
            #if rating == 0:
                # Extract the items the user already rated
            userVector = train_matrix[user, :]
            ratedItems = userVector[userVector.nonzero()]

            # If not empty, calculate average and set as rating for the current item
            if ratedItems.size == 0:
                itemAvg = 0
            else:
                itemAvg = ratedItems.mean()
            predictionMatrix[user, item] = itemAvg

            # report progress every 100 users
            if (user % 100 == 0 and item == 1):
                print ("calculated %d users" % (user,))

        ###########         end         ###########
        return predictionMatrix
    
    @staticmethod
    def popularity(train_matrix, num_users, num_items):
        """
            INPUT:
                train_matrix: 2D numpy array.
                num_users: int. Number of Users.
                num_items: int. Number of Items.
            OUTPUT:
                predictionMatrix: 2D numpy array.
                
            NOTE: see where something very similar is done in the lab in function 'predictByPopularity'    
        """
   
        # Initialize the predicted rating matrix with zeros
        predictionMatrix = np.zeros((num_users, num_items))

        # Define function for converting 1-5 rating to 0/1 (like / don't like)
        vf = np.vectorize(lambda x: 1 if x >= 4 else 0)

        # For every item calculate the number of people liked (4-5) divided by the number of people that rated
        itemPopularity = np.zeros((num_items))
        for item in range(num_items):
            numOfUsersRated = len(train_matrix[:, item].nonzero()[0])
            numOfUsersLiked = len(vf(train_matrix[:, item]).nonzero()[0])
            if numOfUsersRated == 0:
                itemPopularity[item] = 0
            else:
                itemPopularity[item] = numOfUsersLiked/numOfUsersRated

        for (user,item), rating in np.ndenumerate(train_matrix):
            # Predict rating for every item that wasn't ranked by the user (rating == 0)
            #if rating == 0:
            predictionMatrix[user, item] = itemPopularity[item]

            # report progress every 100 users
            if (user % 100 == 0 and item == 1):
                print ("calculated %d users" % (user,))

                  
        ###########         end         ###########
        return predictionMatrix    
    
    def predict_all(self, train_df, num_users, num_items):
        
        train_matrix = self.processor(train_df, num_users, num_items)
        self.__model = self.method(train_matrix, num_users, num_items)
        
    def evaluate_test(self, test_df, copy=False):
        
        if copy:
            prediction = test_df.copy()
        else:
            prediction = test_df
            
        prediction[self.pred_column_name] = np.nan
        
        for (index, 
             userID, 
             itemID) in tqdm(prediction[['userID','itemID']].itertuples()):
            prediction.loc[index, self.pred_column_name] = self.__model[userID-1, itemID-1]

        return prediction
        
    def getModel(self):
        """
            return predicted user-item matrix
        """
        return self.__model
    
    def getPredColName(self):
        """
            return prediction column name
        """
        return self.pred_column_name
    
    def reset(self):
        """
            reuse the instance of the class by removing model
        """
        try:
            self.model = None
        except:
            print("You don not have model..")

In [8]:
popularity_recsys = BaseLineRecSys('popularity')

In [9]:
popularity_recsys.predict_all(rating_df, num_users, num_items)

calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


In [10]:
x = popularity_recsys.getModel()

In [11]:
np.all(x<=1)

True

In [12]:
popularity_recsys.evaluate_test(rating_df,copy=True).head()

100000it [00:58, 1715.90it/s]


,userID,itemID,rating,timestamp,popularity
0,196,242,3,881250949,0.760684
1,186,302,3,891717742,0.804714
2,22,377,1,878887116,0.076923
3,244,51,2,880606923,0.555556
4,166,346,1,886397596,0.611111


In [13]:
average_user_rating_recsys = BaseLineRecSys('useraverage')

In [14]:
average_user_rating_recsys.predict_all(rating_df, num_users, num_items)

calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


In [15]:
average_user_rating_recsys.getModel()

array([[3.61029412, 3.61029412, 3.61029412, ..., 3.61029412, 3.61029412,
        3.61029412],
       [3.70967742, 3.70967742, 3.70967742, ..., 3.70967742, 3.70967742,
        3.70967742],
       [2.7962963 , 2.7962963 , 2.7962963 , ..., 2.7962963 , 2.7962963 ,
        2.7962963 ],
       ...,
       [4.04545455, 4.04545455, 4.04545455, ..., 4.04545455, 4.04545455,
        4.04545455],
       [4.26582278, 4.26582278, 4.26582278, ..., 4.26582278, 4.26582278,
        4.26582278],
       [3.41071429, 3.41071429, 3.41071429, ..., 3.41071429, 3.41071429,
        3.41071429]])

In [16]:
average_user_rating_recsys.evaluate_test(rating_df,copy=True).head()

100000it [00:49, 2015.05it/s]


,userID,itemID,rating,timestamp,useraverage
0,196,242,3,881250949,3.615385
1,186,302,3,891717742,3.413043
2,22,377,1,878887116,3.351562
3,244,51,2,880606923,3.651261
4,166,346,1,886397596,3.550000


### 2A

In [17]:
class SimBasedRecSys(object):

    def __init__(self, base, method, processor=dataPreprocessor):
        """
            base: string. From ['user', 'item']. User-based Similarity or Item-based
            method: string. From ['cosine', 'euclidean', 'somethingelse']
            processor: function name. dataPreprocessor by default
        """
        self.base = base
        self.method_name = method
        self.method = self._getMethod(self.method_name)
        self.processor = processor
        self.pred_column_name = self.base+'-'+self.method_name
    
    def _getMethod(self, method_name):
        """
            Don't change this
        """
        switcher = {
            'cosine': self.cosine,
            'euclidean': self.euclidean,
            'somethingelse': self.somethingelse,
        }
        
        return switcher[method_name]
    
    @staticmethod
    def cosine(matrix):
        """
            cosine similarity
        """
        similarity_matrix = 1 - pairwise_distances(matrix, metric='cosine')
        return similarity_matrix
    
    @staticmethod
    def euclidean(matrix):
        """
            euclidean similarity
        """
        ########### your code goes here ###########

        similarity_matrix = 1 /(1 + pairwise_distances(matrix, metric='euclidean'))
    
        ###########         end         ###########    
        
        return similarity_matrix
    
    @staticmethod
    def somethingelse(matrix):
        """
            manhattan? or super-natural intuition similarity
        """
        ########### your code goes here ###########
    
        similarity_matrix = 1 /(1 + pairwise_distances(matrix, metric='manhattan'))
    
    
        ###########         end         ###########        
        return similarity_matrix
        
    def predict_all(self, train_df, num_users, num_items):
        """
            INPUT: 
                data: pandas DataFrame. columns=['userID', 'itemID', 'rating'...]
                num_row: scalar. number of users
                num_col: scalar. number of items
            OUTPUT:
                no return... this method assigns the result to self.model
            
            NOTES:
                self.__model should contain predictions for *all* user and items
                (don't worry about predicting for observed (user,item) pairs,
                 since we won't be using these predictions in the evaluation)
                (see code in for an efficient vectorized example)
        """
        train_matrix = self.processor(train_df, num_users, num_items)
        
        if self.base == 'user':
            ########### your code goes here ###########

            # Initialize the predicted rating matrix with zeros
            temp_matrix = np.zeros(train_matrix.shape)
            temp_matrix[train_matrix.nonzero()] = 1
            uu_similarity = self.method(train_matrix)
    #         if k is not None:
    #             uu_similarity = kNearestNeighbor(uu_similarity, k)
            normalizer = np.matmul(uu_similarity, temp_matrix)
            normalizer[normalizer == 0] = 1e-5
            predictionMatrix = np.matmul(uu_similarity, train_matrix)/normalizer
            #predictionMatrix[temp_matrix.nonzero()] = 0
            #Cold start
            
            
            normatempsum = np.sum(temp_matrix, axis=1)
            normatempsum[normatempsum == 0] = 1e-5
            useraverage = np.sum(train_matrix, axis=1)/normatempsum
            columns = np.sum(predictionMatrix, axis=0)
            predictionMatrix[:, columns==0] = predictionMatrix[:, columns==0] + np.expand_dims(useraverage, axis=1)

            self.__model = predictionMatrix  
                      
            ###########         end         ###########
            
        elif self.base == 'item':
            ########### your code goes here ###########
            
            train_matrix = train_matrix.transpose()    

            # Initialize the predicted rating matrix with zeros
            temp_matrix = np.zeros(train_matrix.shape)
            temp_matrix[train_matrix.nonzero()] = 1
            uu_similarity = self.method(train_matrix)
    #         if k is not None:
    #             uu_similarity = kNearestNeighbor(uu_similarity, k)
            normalizer = np.matmul(uu_similarity, temp_matrix)
            normalizer[normalizer == 0] = 1e-5
            predictionMatrix = np.matmul(uu_similarity, train_matrix)/normalizer
            #predictionMatrix[temp_matrix.nonzero()] = 0
            #Cold start
            
            
            normatempsum = np.sum(temp_matrix, axis=1)
            normatempsum[normatempsum == 0] = 1e-5
            useraverage = np.sum(train_matrix, axis=1)/normatempsum
            columns = np.sum(predictionMatrix, axis=0)
            predictionMatrix[:, columns==0] = predictionMatrix[:, columns==0] + np.expand_dims(useraverage, axis=1)

            self.__model = predictionMatrix.transpose()
            
          
            ###########         end         ###########
        else:
            print('No other option available')
        
    def evaluate_test(self, test_df, copy=False):
        """
            INPUT:
                data: pandas DataFrame. columns=['userID', 'itemID', 'rating'...]
            OUTPUT:
                predictions:  pandas DataFrame. 
                              columns=['userID', 'itemID', 'rating', 'base-method'...]
                              
            NOTE: 1. data can have more columns, but your function should ignore 
                  additional columns.
                  2. 'base-method' depends on your 'base' and 'method'. For example,
                  if base == 'user' and method == 'cosine', 
                  then base-method == 'user-cosine'
                  3. your predictions go to 'base-method' column
        """
        if copy:
            prediction = test_df.copy()
        else:
            prediction = test_df
        prediction[self.pred_column_name] = np.nan
        
        for (index, 
             userID, 
             itemID) in tqdm(prediction[['userID','itemID']].itertuples()):
            prediction.loc[index, self.pred_column_name] = self.__model[userID-1, itemID-1]
    
        return prediction
    
    def getModel(self):
        """
            return predicted user-item matrix
        """
        return self.__model
    
    def getPredColName(self):
        """
            return prediction column name
        """
        return self.pred_column_name
    
    def reset(self):
        """
            reuse the instance of the class by removing model
        """
        try:
            self.model = None
        except:
            print("You do not have model..")

In [18]:
I = [[1,1],[5,5]]
SimBasedRecSys.cosine(I)

array([[1., 1.],
       [1., 1.]])

In [19]:
I = np.eye(3)
SimBasedRecSys.cosine(I)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [20]:
SimBasedRecSys.euclidean(I)

array([[1.        , 0.41421356, 0.41421356],
       [0.41421356, 1.        , 0.41421356],
       [0.41421356, 0.41421356, 1.        ]])

In [21]:
SimBasedRecSys.somethingelse(I)

array([[1.        , 0.33333333, 0.33333333],
       [0.33333333, 1.        , 0.33333333],
       [0.33333333, 0.33333333, 1.        ]])

Analysis: 
* Cosine works better than Euclidean mainly due to length normalization.
* But in our case we also normalized values of Euclidean so both are having overlap in confidence intervals for RMSE.
* For example If I have two users who rated (4,4) and (5,5) two movies they can still be similar because each user has different relative perspectives both of them didnt hate the movie. So in this case cosine works better than Euclidean.
* For example If I have two user who rated (1,1) and (5,5) two movies. Cosine says they are similar but clearly they are not.
* From our results in Q4
'user-euclidean': [[1.0319953645887305, 1.013706565892023, 1.031268958300787], 'item-euclidean': [[1.0452460304884215, 1.0157114777932958, 1.0413212192481176]}
'user-cosine': [[1.026449013124381, 1.009013080226148, 1.0256951630950135], 'item-cosine': [[1.0377631264364244, 1.0068242686250732, 1.0333415315874226]}


### 2B

Manhattan is choosen because it calculates the distance of two vectors by absolute differences. It is similar to Euclidean because we are normalizing both of them (similar results checked in bottom). In case of manhattan it is more robust and it requires less calculations.


### 3A

In [22]:
user_cosine_recsys = SimBasedRecSys('user','cosine')

In [23]:
item_cosine_recsys = SimBasedRecSys('item','cosine')

In [24]:
user_cosine_recsys.predict_all(rating_df, num_users, num_items)

In [25]:
user_cosine_recsys.getModel()

array([[3.89911175, 3.19022667, 3.0261129 , ..., 2.        , 3.        ,
        3.        ],
       [3.84034456, 3.17139889, 2.92626717, ..., 2.        , 3.        ,
        3.        ],
       [3.87104065, 3.12823798, 3.03250708, ..., 2.        , 3.        ,
        3.        ],
       ...,
       [3.90754645, 3.20227238, 3.05776201, ..., 2.        , 3.        ,
        3.        ],
       [3.91100649, 3.21591021, 2.98854017, ..., 2.        , 3.        ,
        3.        ],
       [3.91593122, 3.24268207, 3.08255897, ..., 0.        , 3.        ,
        3.        ]])

In [26]:
user_cosine_recsys.evaluate_test(rating_df,copy=True).head()

100000it [01:52, 884.99it/s]


,userID,itemID,rating,timestamp,user-cosine
0,196,242,3,881250949,4.025213
1,186,302,3,891717742,4.142828
2,22,377,1,878887116,1.922080
3,244,51,2,880606923,3.431884
4,166,346,1,886397596,3.424963


### 3B

In [27]:
class CrossValidation(object):
    def __init__(self, metric, data_path=path):
        """
            INPUT:
                metric: string. from['RMSE','P@K','R@K']
        """
        self.folds = self._getData(path)
        self.metric_name = metric
        self.metric = self._getMetric(self.metric_name)
        
    def _getMetric(self, metric_name):
        """
            Don't change this
        """
        switcher = {
            'RMSE': self.rmse,
            'P@K': self.patk,
            'R@K': self.ratk,
        }
        
        return switcher[metric_name]
    
    @staticmethod
    def rmse(data, k, num_users, num_items, pred, true='rating'):
        """
            data: pandas DataFrame. 
            pred: string. Column name that corresponding to the prediction
            true: string. Column name that corresponding to the true rating
        """
        return sqrt(mean_squared_error(data[pred], data[true]))
    
    # Precision at k
    def patk(self, data, k, num_users, num_items, pred, true='rating'):
        """
            data: pandas DataFrame. 
            k: top-k items retrived
            pred: string. Column name that corresponding to the prediction
            true: string. Column name that corresponding to the true rating
        """
        prediction = self.getMatrix(data, num_users, num_items, pred)
        testSet =  self.getMatrix(data, num_users, num_items, true)
    
        # Initialize sum and count vars for average calculation
        sumPrecisions = 0
        countPrecisions = 0

        # Define function for converting 1-5 rating to 0/1 (like / don't like)
        vf = np.vectorize(lambda x: 1 if x >= 4 else 0)

        for userID in range(num_users):
            # Pick top K based on predicted rating
            userVector = prediction[userID,:]
            topK = nlargest(k, range(len(userVector)), userVector.take)

            # Convert test set ratings to like / don't like
            userTestVector = vf(testSet[userID,:]).nonzero()[0]

            # Calculate precision
            precision = float(len([item for item in topK if item in userTestVector]))/len(topK)

            # Update sum and count
            sumPrecisions += precision
            countPrecisions += 1

        # Return average P@k
        return float(sumPrecisions)/countPrecisions
    
    # Recall at k
    def ratk(self, data, k, num_users, num_items, pred, true='rating'):
        """
            data: pandas DataFrame. 
            k: top-k items relevant
            pred: string. Column name that corresponding to the prediction
            true: string. Column name that corresponding to the true rating
        """
        prediction = self.getMatrix(data, num_users, num_items, pred)
        testSet =  self.getMatrix(data, num_users, num_items, true)
        # Initialize sum and count vars for average calculation
        sumRecalls = 0
        countRecalls = 0

        # Define function for converting 1-5 rating to 0/1 (like / don't like)
        vf = np.vectorize(lambda x: 1 if x >= 4 else 0)

        for userID in range(num_users):
            # Pick top K based on predicted rating
            userVector = prediction[userID,:]
            topK = nlargest(k, range(len(userVector)), userVector.take)

            # Convert test set ratings to like / don't like
            userTestVector = vf(testSet[userID,:]).nonzero()[0]

            # Ignore user if has no ratings in the test set
            if (len(userTestVector) == 0):
                continue

            # Calculate recall
            recall = float(len([item for item in topK if item in userTestVector]))/len(userTestVector)

            # Update sum and count
            sumRecalls += recall
            countRecalls += 1

        # Return average R@k
        return float(sumRecalls)/countRecalls
    
    @staticmethod
    def getMatrix(rating_df, num_users, num_items, column_name):
        matrix = np.zeros((num_users, num_items))
    
        for (index, userID, itemID, value) in rating_df[['userID','itemID', column_name]].itertuples():
            matrix[userID-1, itemID-1] = value
            
        return matrix
    
    @staticmethod
    def _getData(data_path):
        """
            Don't change this function
        """
        folds = []
        data_types = ['u{0}.base','u{0}.test']
        for i in range(1,6):
            train_set = getData(data_path, data_types[0].format(i))
            test_set = getData(data_path, data_types[1].format(i))
            folds.append([train_set, test_set])
        return folds
    
    def run(self, algorithms, num_users, num_items, k=1):
        """
            5-fold cross-validation
            algorithms: list. a list of algorithms. 
                        eg: [user_cosine_recsys, item_euclidean_recsys]
        """
        
        scores = {}
        for algorithm in algorithms:
            print('Processing algorithm {0}'.format(algorithm.getPredColName()))
            fold_scores = []
            for fold in self.folds:
                algorithm.reset()
                algorithm.predict_all(fold[0], num_users, num_items)
                prediction = algorithm.evaluate_test(fold[1])
                pred_col = algorithm.getPredColName()
                fold_scores.append(self.metric(prediction, k, num_users, num_items, pred_col))
                
            mean = np.mean(fold_scores)
            ci_low, ci_high = stats.t.interval(0.95, len(fold_scores)-1, loc=mean, scale=stats.sem(fold_scores))
            scores[algorithm.getPredColName()] = [fold_scores, mean, ci_low, ci_high]
            
        results = scores    
    
        return results

In [28]:
# How to use CrossValidation Class?
# 1. gather your algorithms in previous steps.
algorithm_instances = [popularity_recsys, 
                       average_user_rating_recsys, 
                       user_cosine_recsys,
                       item_cosine_recsys]

In [29]:
# 2. Instantiate a CrossValidation instance and assign the measurement that you want to use
# RMSE, P@K, R@K
# Precision at K in this example
cv_patk = CrossValidation('RMSE')

In [30]:
# 3. Run CV by giving:
#    1> algorithms just gathered
#    2> number of users in the full dataset
#    3> number of items in the full dataset
#    4> precision or recall at K need a K value, so k=5 means precision at 5 in this example
# Results include independent results from 5 folds, their mean, and confidence interval.
cv_patk.run(algorithm_instances, num_users, num_items,k=5)

Processing algorithm popularity
calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


20000it [00:06, 3329.18it/s]


calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


20000it [00:05, 3413.57it/s]


calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


...

Processing algorithm user-cosine


20000it [00:05, 3348.95it/s]
20000it [00:05, 3434.38it/s]
20000it [00:05, 3410.78it/s]
20000it [00:05, 3349.49it/s]
20000it [00:05, 3451.16it/s]


Processing algorithm item-cosine


20000it [00:05, 3354.53it/s]
20000it [00:05, 3436.43it/s]
20000it [00:05, 3438.37it/s]
20000it [00:06, 3332.37it/s]
20000it [00:05, 3422.64it/s]


{'popularity': [[3.177941281084362,
   3.1750480150769977,
   3.147474655005899,
   3.146164503024159,
   3.1488360007536382],
  3.1590928909890112,
  3.139292746995387,
  3.1788930349826354],
 'useraverage': [[1.0629951276561334,
   1.0467467492319966,
   1.0328964562995389,
   1.0366575971298078,
   1.0392923504800367],
  1.0437176561595025,
  1.0289303496379316,
  1.0585049626810734],
 'user-cosine': [[1.026449013124381,
   1.0214387664779507,
   1.0132940326457187,
   1.0094003999022947,
   1.0161883961525586],
  1.0173541216605808,
  1.009013080226148,
  1.0256951630950135],
 'item-cosine': [[1.0377631264364244,
   1.0207280585350078,
   1.0101820660011798,
   1.0136832839209695,
   1.0180579656376574],
  1.020082900106248,
  1.0068242686250732,
  1.0333415315874226]}

In [31]:
q3df = dataPreprocessor(rating_df, num_users, num_items)

In [32]:
temp_matrix = np.zeros(q3df.shape)
temp_matrix[q3df.nonzero()] = 1 
sumperrow = np.sum(temp_matrix, axis=1)
averagenumberofratingsperuser = (sumperrow.sum())/943
averagenumberofratingsperitem = (sumperrow.sum())/1682

In [33]:
averagenumberofratingsperitem

59.45303210463734

In [34]:
averagenumberofratingsperuser

106.04453870625663

Analysis: 
* The following values are RMSE average, low CI and high CI
'item-cosine': [ 1.020, 1.006, 1.033,
'user-cosine': [ 1.017, 1.009, 1.025,

* If we see the RMSE results the user is performing slightly better because in our case the average number of ratings per user 106 is more than the average number of ratings per item 59. So while using similarity metrics in case of users we have more rich vectors to find out the similar ones. so better predictions.

### 4A

In [35]:
item_cosine_recsys = SimBasedRecSys('item','cosine')

In [36]:
user_cosine_recsys = SimBasedRecSys('user','cosine')

In [37]:
# 1. gather your algorithms in previous steps.
algorithm_instances = [popularity_recsys, 
                       average_user_rating_recsys, 
                       user_cosine_recsys,
                       item_cosine_recsys]

In [38]:
cv_patk = CrossValidation('RMSE')

In [39]:
cv_patk.run(algorithm_instances, num_users, num_items,k=5)

Processing algorithm popularity
calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


20000it [00:06, 3295.96it/s]


calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


20000it [00:06, 3232.34it/s]


calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


...

Processing algorithm user-cosine


20000it [00:06, 3316.56it/s]
20000it [00:05, 3449.75it/s]
20000it [00:05, 3441.06it/s]
20000it [00:06, 3317.58it/s]
20000it [00:05, 3449.48it/s]


Processing algorithm item-cosine


20000it [00:06, 3289.40it/s]
20000it [00:05, 3430.21it/s]
20000it [00:05, 3428.52it/s]
20000it [00:06, 3272.64it/s]
20000it [00:05, 3467.53it/s]


{'popularity': [[3.177941281084362,
   3.1750480150769977,
   3.147474655005899,
   3.146164503024159,
   3.1488360007536382],
  3.1590928909890112,
  3.139292746995387,
  3.1788930349826354],
 'useraverage': [[1.0629951276561334,
   1.0467467492319966,
   1.0328964562995389,
   1.0366575971298078,
   1.0392923504800367],
  1.0437176561595025,
  1.0289303496379316,
  1.0585049626810734],
 'user-cosine': [[1.026449013124381,
   1.0214387664779507,
   1.0132940326457187,
   1.0094003999022947,
   1.0161883961525586],
  1.0173541216605808,
  1.009013080226148,
  1.0256951630950135],
 'item-cosine': [[1.0377631264364244,
   1.0207280585350078,
   1.0101820660011798,
   1.0136832839209695,
   1.0180579656376574],
  1.020082900106248,
  1.0068242686250732,
  1.0333415315874226]}

In [40]:
cv_patk = CrossValidation('P@K')

In [41]:
cv_patk.run(algorithm_instances, num_users, num_items,k=5)

Processing algorithm popularity
calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


20000it [00:06, 3281.94it/s]


calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


20000it [00:05, 3414.01it/s]


calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


...

Processing algorithm user-cosine


20000it [00:05, 3421.53it/s]
20000it [00:05, 3401.18it/s]
20000it [00:05, 3431.71it/s]
20000it [00:05, 3343.35it/s]
20000it [00:05, 3460.47it/s]


Processing algorithm item-cosine


20000it [00:06, 3316.23it/s]
20000it [00:05, 3410.82it/s]
20000it [00:06, 3322.89it/s]
20000it [00:05, 3430.33it/s]
20000it [00:05, 3341.18it/s]


{'popularity': [[0.36924708377518656,
   0.4965005302226948,
   0.6152704135737019,
   0.6426299045599162,
   0.6292682926829279],
  0.5505832449628855,
  0.40544114481568705,
  0.6957253451100839],
 'useraverage': [[0.30604453870625714,
   0.4305408271474029,
   0.5321314952279973,
   0.5520678685047737,
   0.5474019088016986],
  0.4736373276776259,
  0.3419993013451059,
  0.6052753540101459],
 'user-cosine': [[0.37179215270413657,
   0.503923647932133,
   0.621633085896077,
   0.6483563096500541,
   0.6335100742311777],
  0.5558430540827157,
  0.40959849499983714,
  0.7020876131655943],
 'item-cosine': [[0.34316012725344736,
   0.483563096500532,
   0.6021208907741271,
   0.6248144220572649,
   0.6074231177094392],
  0.5322163308589621,
  0.3837005215009889,
  0.6807321402169354]}

In [42]:
cv_patk = CrossValidation('R@K')

In [43]:
cv_patk.run(algorithm_instances, num_users, num_items,k=5)

Processing algorithm popularity
calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


20000it [00:05, 3430.70it/s]


calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


20000it [00:06, 3310.37it/s]


calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


...

Processing algorithm user-cosine


20000it [00:05, 3421.28it/s]
20000it [00:06, 3281.53it/s]
20000it [00:05, 3449.26it/s]
20000it [00:06, 3298.18it/s]
20000it [00:05, 3440.98it/s]


Processing algorithm item-cosine


20000it [00:05, 3358.31it/s]
20000it [00:05, 3418.78it/s]
20000it [00:05, 3432.82it/s]
20000it [00:05, 3378.12it/s]
20000it [00:05, 3420.55it/s]


{'popularity': [[0.3466588624187514,
   0.4274468698270901,
   0.5269205125667804,
   0.5518738761026849,
   0.5674793185065369],
  0.4840758878843688,
  0.3671373629798323,
  0.6010144127889052],
 'useraverage': [[0.30505841002027845,
   0.39554692074366876,
   0.48030412192442223,
   0.5045885853815734,
   0.5211179870422066],
  0.44132320502242983,
  0.32931026359142457,
  0.5533361464534351],
 'user-cosine': [[0.34778041993806913,
   0.4314035774468209,
   0.5293633772333985,
   0.5553818201403046,
   0.5674144230096255],
  0.4862687235536437,
  0.3694473610987218,
  0.6030900860085656],
 'item-cosine': [[0.3277711938444533,
   0.4237782250680911,
   0.5191391022223312,
   0.5448659224612776,
   0.5593011306991799],
  0.4749711148590666,
  0.35357317503649865,
  0.5963690546816346]}

Q4) Analysis:

(A) 
For each of the metrics the three values are Average, Low CI and high CI.
             Ave LowCI HIgh CI


* RMSE {'item-cosine': [ 1.020, 1.006, 1.033,
'popularity': [ 3.159, 3.139, 3.178,
'user-cosine': [ 1.017, 1.009, 1.025,
'useraverage': [ 1.043, 1.028, 1.058,}
P@K
{'item-cosine': [ 0.532, 0.383, 0.680,
'popularity': [ 0.550, 0.405, 0.695,
'user-cosine': [ 0.555, 0.409, 0.702,
'useraverage': [ 0.473, 0.341, 0.605]}
R@K
{'item-cosine': [ 0.474, 0.353, 0.596],
'popularity': [ 0.484, 0.367, 0.601,
'user-cosine': [ 0.486, 0.369, 0.603,
'useraverage': [ 0.441, 0.329, 0.553}



(B)
Some baselines cannot be evaluated with some metrics? Which ones and why?

* For User Average This predicts the same average rating for all the movies for a user (for the movies there are no ratings). In this case rank based metrics P@k and R@k cannot be evaluated. For popularity, We cannot use RMSE for popularity becuase the popularity values in this case are between 0 to 1 (because we are not taking the exact rating for calculation we have only taken if the user liked(rating > 4) or not) but the test ratings are from 1 to 5. so RMSE cannot be evaluated



(C)
What is the best algorithm for each of RMSE, P@k, and R@k? Can you explain why this may be?

* P@k and R@k the evaluations are based on ranking (we need to recommend most relavant to each user). For better ranking the similarity metrics are most suitable because the popularity and average are not persnolized to each user. From the results we can say that user cosine average performed slighlty better in both P@K and R@K. For RMSE cosine similarity measures work better becuase the similarity measures predict the rating based on ratings of most similar users or items. We can also see that in case of RMSE similarity measure have better performed.



(D)
Does good performance on RMSE imply good performance on ranking metrics and vice versa? Why / why not?

* No good performance of RMSE doesn't imply good performace on ranking metrics. Becuase RMSE is calulated for all the user ratings wheter he liked it or not irrespective of their rankings. So good RMSE may come at expense of bad ranking of top K. Similarly, in case of ranking metrics we are only concerned about ranking in top k and have no clue about others. So good performance of ranking doesn't necasarily mean good performance of RMSE.

In [44]:
# calculating Eculidean to know which is better
item_cosine_recsys = SimBasedRecSys('item','euclidean')

In [45]:
user_cosine_recsys = SimBasedRecSys('user','euclidean')

In [46]:
# 1. gather your algorithms in previous steps.
algorithm_instances = [user_cosine_recsys, item_cosine_recsys]

In [47]:
cv_patk = CrossValidation('RMSE')

In [48]:
cv_patk.run(algorithm_instances, num_users, num_items,k=5)

Processing algorithm user-euclidean


20000it [00:05, 3393.29it/s]
20000it [00:05, 3397.10it/s]
20000it [00:06, 3247.34it/s]
20000it [00:05, 3430.09it/s]
20000it [00:06, 3323.14it/s]


Processing algorithm item-euclidean


20000it [00:05, 3412.78it/s]
20000it [00:05, 3420.65it/s]
20000it [00:06, 3239.82it/s]
20000it [00:05, 3411.60it/s]
20000it [00:06, 3289.98it/s]


{'user-euclidean': [[1.0319953645887305,
   1.0272960502657182,
   1.017739206917733,
   1.0146850193650487,
   1.0207231693447965],
  1.0224877620964052,
  1.0137065658920232,
  1.0312689583007872],
 'item-euclidean': [[1.0452460304884215,
   1.0315283051917867,
   1.0203644555365832,
   1.0215306724762894,
   1.023912278910453],
  1.0285163485207067,
  1.0157114777932958,
  1.0413212192481176]}

In [49]:
user_cosine_recsys = SimBasedRecSys('user','euclidean')
item_cosine_recsys = SimBasedRecSys('item','euclidean')

In [50]:
# 1. gather your algorithms in previous steps.
algorithm_instances = [user_cosine_recsys, item_cosine_recsys]

In [51]:
cv_patk = CrossValidation('RMSE')

In [52]:
cv_patk.run(algorithm_instances, num_users, num_items,k=5)

Processing algorithm user-euclidean


20000it [00:05, 3419.81it/s]
20000it [00:05, 3411.26it/s]
20000it [00:06, 3288.88it/s]
20000it [00:06, 3266.17it/s]
20000it [00:06, 3113.72it/s]


Processing algorithm item-euclidean


20000it [00:05, 3392.81it/s]
20000it [00:05, 3421.34it/s]
20000it [00:06, 3270.66it/s]
20000it [00:05, 3410.89it/s]
20000it [00:06, 3239.66it/s]


{'user-euclidean': [[1.0319953645887305,
   1.0272960502657182,
   1.017739206917733,
   1.0146850193650487,
   1.0207231693447965],
  1.0224877620964052,
  1.0137065658920232,
  1.0312689583007872],
 'item-euclidean': [[1.0452460304884215,
   1.0315283051917867,
   1.0203644555365832,
   1.0215306724762894,
   1.023912278910453],
  1.0285163485207067,
  1.0157114777932958,
  1.0413212192481176]}

### 5A

In [53]:
q5df = dataPreprocessor(rating_df, num_users, num_items)
q5df

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]], dtype=int8)

In [54]:
fieldsMovies = ['movieID', 'movieTitle', 'releaseDate', 'videoReleaseDate', 'IMDbURL', 'unknown', 'action', 'adventure',
          'animation', 'childrens', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'filmNoir', 'horror',
          'musical', 'mystery', 'romance','sciFi', 'thriller', 'war', 'western']
moviesDF = pd.read_csv(os.path.join(path, 'u.item'), sep='|', names=fieldsMovies, encoding='latin-1')
moviesDF.head()

,movieID,movieTitle,releaseDate,videoReleaseDate,IMDbURL,unknown,action,adventure,animation,childrens,...,fantasy,filmNoir,horror,musical,mystery,romance,sciFi,thriller,war,western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [55]:
moviesDF[1100:1300]

,movieID,movieTitle,releaseDate,videoReleaseDate,IMDbURL,unknown,action,adventure,animation,childrens,...,fantasy,filmNoir,horror,musical,mystery,romance,sciFi,thriller,war,western
1100,1101,Six Degrees of Separation (1993),01-Jan-1993,NaN,http://us.imdb.com/M/title-exact?Six%20Degrees...,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1101,1102,Two Much (1996),01-Jan-1996,NaN,http://us.imdb.com/M/title-exact?Two%20Much%20...,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1102,1103,Trust (1990),01-Jan-1990,NaN,http://us.imdb.com/Title?Trust+(1990),0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1103,1104,C'est arrivé près de chez vous (1992),01-Jan-1992,NaN,http://us.imdb.com/M/title-exact?C%27est%20arr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1104,1105,Firestorm (1998),09-Jan-1998,NaN,http://us.imdb.com/M/title-exact?imdb-title-12...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1105,1106,"Newton Boys, The (1998)",14-Mar-1998,NaN,"http://us.imdb.com/Title?Newton+Boys,+The+(1998)",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1106,1107,Beyond Rangoon (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Beyond%20Rang...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1107,1108,Feast of July (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Feast%20of%20...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1108,1109,Death and the Maiden (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Death%20and%2...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1109,1110,Tank Girl (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Tank%20Girl%2...,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0


In [56]:
listindex = [109,223,526,873,1298]

In [57]:
listmovieid = [110,224,527,874,1299]

In [58]:
trainSet = q5df
trainSet = trainSet.transpose()  
uu_similarity = 1 - pairwise_distances(trainSet, metric='cosine')
uu_similarity

array([[1.        , 0.40238218, 0.33024479, ..., 0.        , 0.04718307,
        0.04718307],
       [0.40238218, 1.        , 0.27306918, ..., 0.        , 0.07829936,
        0.07829936],
       [0.33024479, 0.27306918, 1.        , ..., 0.        , 0.        ,
        0.09687505],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.04718307, 0.07829936, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.04718307, 0.07829936, 0.09687505, ..., 0.        , 0.        ,
        1.        ]])

In [59]:
final = np.argsort(uu_similarity, axis=1, kind='quicksort', order=None)[:,1676:1682]
final

array([[ 404,  116,  120,  180,   49,    0],
       [  61,  402,  384,  160,  232,    1],
       [ 249,   32,   41,  762,  409,    2],
       ...,
       [ 909, 1430, 1394, 1677, 1678, 1679],
       [1405, 1422, 1621, 1350, 1671, 1680],
       [ 556,  592, 1596, 1334,  766, 1681]], dtype=int64)

In [60]:
listmovieid = [110,224,527,874,1299]
listindex = [109,223,526,873,1298]
moviesDF.iloc[109,1]

'Operation Dumbo Drop (1995)'

In [61]:
for i in listindex:
    movielist = final[i]
    print("Top similar movies to")
    print(moviesDF.iloc[i,1])
    print("are")

    for j in movielist:
        print(moviesDF.iloc[j,1])
    print("\n")

Top similar movies to
Operation Dumbo Drop (1995)
are
Corrina, Corrina (1994)
Made in America (1993)
Santa Clause, The (1994)
Angels in the Outfield (1994)
Junior (1994)
Operation Dumbo Drop (1995)


Top similar movies to
Ridicule (1996)
are
Looking for Richard (1996)
Horseman on the Roof, The (Hussard sur le toit, Le) (1995)
Big Night (1996)
Little Odessa (1994)
Cold Comfort Farm (1995)
Ridicule (1996)


Top similar movies to
Gandhi (1982)
are
Casablanca (1942)
Raiders of the Lost Ark (1981)
Forrest Gump (1994)
One Flew Over the Cuckoo's Nest (1975)
Amadeus (1984)
Gandhi (1982)


Top similar movies to
Career Girls (1997)
are
In the Company of Men (1997)
Ice Storm, The (1997)
Ulee's Gold (1997)
Kicked in the Head (1997)
Steel (1997)
Career Girls (1997)


Top similar movies to
Penny Serenade (1941)
are
Love in the Afternoon (1957)
Laura (1944)
Band Wagon, The (1953)
Gay Divorcee, The (1934)
Charade (1963)
Penny Serenade (1941)




### 5B

If group of items are similar it means their item vectors are similar. This happens when same user ranked all of the items.
In this case the magnitude of their rating doesn't matter becuase we are using cosine metric. only thing which matters if the same users have rated the items or not. For this to happen there could be many reasons like all the movies belonging to a particular genre or all of them have same actor, genre or director.

Justify : 
Top similar movies to Career Girls (1997)
are In the Company of Men (1997)
Ice Storm, The (1997)
Ulee's Gold (1997)
Kicked in the Head (1997)
Steel (1997)
Career Girls (1997)

All of them have same genre "DRAMA" and all of them are released in same year. In fact the movies all released in same year. So DRAMA lovers of 1990's must have rated all the movies.


### 6A

In [62]:
q6df = dataPreprocessor(rating_df, num_users, num_items)

In [63]:
temp_matrix = np.zeros(q6df.shape)
temp_matrix[q6df.nonzero()] = 1 
userratingsnumber = np.sum(temp_matrix, axis=1)
#userratingsnumber.shape

In [64]:
import matplotlib.pyplot as plt
plt.hist(userratingsnumber, bins=50)
plt.title("Histogram with 'auto' bins")
plt.show()
#userratingsnumber

<Figure size 640x480 with 1 Axes>

In [65]:
np.percentile(userratingsnumber,75)

148.0

In [66]:
thresholdbelow = np.argwhere(userratingsnumber <= 148).flatten()
len(thresholdbelow)

709

In [67]:
thresholdabove = np.argwhere(userratingsnumber > 148).flatten()
len(thresholdabove)

234

In [68]:
class CrossValidationq6above(object):
    def __init__(self, metric, data_path=path):
        """
            INPUT:
                metric: string. from['RMSE','P@K','R@K']
        """
        self.folds = self._getData(path)
        self.metric_name = metric
        self.metric = self._getMetric(self.metric_name)
        
    def _getMetric(self, metric_name):
        """
            Don't change this
        """
        switcher = {
            'RMSE': self.rmse,
            'P@K': self.patk,
            'R@K': self.ratk,
        }
        
        return switcher[metric_name]
    
    @staticmethod
    def rmse(data, k, num_users, num_items, pred, true='rating'):
        """
            data: pandas DataFrame. 
            pred: string. Column name that corresponding to the prediction
            true: string. Column name that corresponding to the true rating
        """
        return sqrt(mean_squared_error(data[pred], data[true]))
    
    # Precision at k
    def patk(self, data, k, num_users, num_items, pred, true='rating'):
        """
            data: pandas DataFrame. 
            k: top-k items retrived
            pred: string. Column name that corresponding to the prediction
            true: string. Column name that corresponding to the true rating
        """
        prediction = self.getMatrix(data, num_users, num_items, pred)
        testSet =  self.getMatrix(data, num_users, num_items, true)
    
        # Initialize sum and count vars for average calculation
        sumPrecisions = 0
        countPrecisions = 0

        # Define function for converting 1-5 rating to 0/1 (like / don't like)
        vf = np.vectorize(lambda x: 1 if x >= 4 else 0)

        for userID in range(num_users):
            # Pick top K based on predicted rating
            userVector = prediction[userID,:]
            topK = nlargest(k, range(len(userVector)), userVector.take)

            # Convert test set ratings to like / don't like
            userTestVector = vf(testSet[userID,:]).nonzero()[0]

            # Calculate precision
            precision = float(len([item for item in topK if item in userTestVector]))/len(topK)

            # Update sum and count
            sumPrecisions += precision
            countPrecisions += 1

        # Return average P@k
        return float(sumPrecisions)/countPrecisions
    
    # Recall at k
    def ratk(self, data, k, num_users, num_items, pred, true='rating'):
        """
            data: pandas DataFrame. 
            k: top-k items relevant
            pred: string. Column name that corresponding to the prediction
            true: string. Column name that corresponding to the true rating
        """
        prediction = self.getMatrix(data, num_users, num_items, pred)
        testSet =  self.getMatrix(data, num_users, num_items, true)
        # Initialize sum and count vars for average calculation
        sumRecalls = 0
        countRecalls = 0

        # Define function for converting 1-5 rating to 0/1 (like / don't like)
        vf = np.vectorize(lambda x: 1 if x >= 4 else 0)

        for userID in range(num_users):
            # Pick top K based on predicted rating
            userVector = prediction[userID,:]
            topK = nlargest(k, range(len(userVector)), userVector.take)

            # Convert test set ratings to like / don't like
            userTestVector = vf(testSet[userID,:]).nonzero()[0]

            # Ignore user if has no ratings in the test set
            if (len(userTestVector) == 0):
                continue

            # Calculate recall
            recall = float(len([item for item in topK if item in userTestVector]))/len(userTestVector)

            # Update sum and count
            sumRecalls += recall
            countRecalls += 1

        # Return average R@k
        return float(sumRecalls)/countRecalls
    
    @staticmethod
    def getMatrix(rating_df, num_users, num_items, column_name):
        matrix = np.zeros((num_users, num_items))
    
        for (index, userID, itemID, value) in rating_df[['userID','itemID', column_name]].itertuples():
            matrix[userID-1, itemID-1] = value
            
        return matrix
    
    @staticmethod
    def _getData(data_path):
        """
            Don't change this function
        """
        folds = []
        data_types = ['u{0}.base','u{0}.test']
        for i in range(1,6):
            train_set = getData(data_path, data_types[0].format(i))
            train_set = train_set.loc[train_set['userID'].isin(thresholdabove)]
            
            test_set = getData(data_path, data_types[1].format(i))
            test_set = test_set.loc[test_set['userID'].isin(thresholdabove)]
            
            folds.append([train_set, test_set])
        return folds
    
    def run(self, algorithms, num_users, num_items, k=1):
        """
            5-fold cross-validation
            algorithms: list. a list of algorithms. 
                        eg: [user_cosine_recsys, item_euclidean_recsys]
        """
        
        scores = {}
        for algorithm in algorithms:
            print('Processing algorithm {0}'.format(algorithm.getPredColName()))
            fold_scores = []
            for fold in self.folds:
                algorithm.reset()
                algorithm.predict_all(fold[0], num_users, num_items)
                prediction = algorithm.evaluate_test(fold[1])
                pred_col = algorithm.getPredColName()
                fold_scores.append(self.metric(prediction, k, num_users, num_items, pred_col))
                
            mean = np.mean(fold_scores)
            ci_low, ci_high = stats.t.interval(0.95, len(fold_scores)-1, loc=mean, scale=stats.sem(fold_scores))
            scores[algorithm.getPredColName()] = [fold_scores, mean, ci_low, ci_high]
            
        results = scores    
    
        return results

In [69]:
item_cosine_recsys = SimBasedRecSys('item','cosine')

In [70]:
user_cosine_recsys = SimBasedRecSys('user','cosine')

In [71]:
# 1. gather your algorithms in previous steps.
algorithm_instances_Q6 = [user_cosine_recsys,
                          item_cosine_recsys]

In [72]:
cv_patk = CrossValidationq6above('RMSE')
cv_patk.run(algorithm_instances_Q6, num_users, num_items)

Processing algorithm user-cosine


5690it [00:01, 3613.85it/s]
5970it [00:01, 3570.11it/s]
5067it [00:01, 3649.60it/s]
4833it [00:01, 3637.40it/s]
4869it [00:01, 3618.46it/s]


Processing algorithm item-cosine


5690it [00:01, 3218.61it/s]
5970it [00:01, 3673.40it/s]
5067it [00:01, 3607.05it/s]
4833it [00:01, 3601.00it/s]
4869it [00:01, 3575.23it/s]


{'user-cosine': [[1.0503720301028903,
   1.0760128059562617,
   1.0570055555699578,
   1.054340869987063,
   1.0468077895648276],
  1.0569078102362,
  1.042800757991966,
  1.0710148624804339],
 'item-cosine': [[1.0568322167581496,
   1.058552860154752,
   1.0251820359216575,
   1.0437442042286282,
   1.0722515432038346],
  1.0513125720534044,
  1.029262861402853,
  1.073362282703956]}

In [73]:
class CrossValidationq6below(object):
    def __init__(self, metric, data_path=path):
        """
            INPUT:
                metric: string. from['RMSE','P@K','R@K']
        """
        self.folds = self._getData(path)
        self.metric_name = metric
        self.metric = self._getMetric(self.metric_name)
        
    def _getMetric(self, metric_name):
        """
            Don't change this
        """
        switcher = {
            'RMSE': self.rmse,
            'P@K': self.patk,
            'R@K': self.ratk,
        }
        
        return switcher[metric_name]
    
    @staticmethod
    def rmse(data, k, num_users, num_items, pred, true='rating'):
        """
            data: pandas DataFrame. 
            pred: string. Column name that corresponding to the prediction
            true: string. Column name that corresponding to the true rating
        """
        return sqrt(mean_squared_error(data[pred], data[true]))
    
    # Precision at k
    def patk(self, data, k, num_users, num_items, pred, true='rating'):
        """
            data: pandas DataFrame. 
            k: top-k items retrived
            pred: string. Column name that corresponding to the prediction
            true: string. Column name that corresponding to the true rating
        """
        prediction = self.getMatrix(data, num_users, num_items, pred)
        testSet =  self.getMatrix(data, num_users, num_items, true)
    
        # Initialize sum and count vars for average calculation
        sumPrecisions = 0
        countPrecisions = 0

        # Define function for converting 1-5 rating to 0/1 (like / don't like)
        vf = np.vectorize(lambda x: 1 if x >= 4 else 0)

        for userID in range(num_users):
            # Pick top K based on predicted rating
            userVector = prediction[userID,:]
            topK = nlargest(k, range(len(userVector)), userVector.take)

            # Convert test set ratings to like / don't like
            userTestVector = vf(testSet[userID,:]).nonzero()[0]

            # Calculate precision
            precision = float(len([item for item in topK if item in userTestVector]))/len(topK)

            # Update sum and count
            sumPrecisions += precision
            countPrecisions += 1

        # Return average P@k
        return float(sumPrecisions)/countPrecisions
    
    # Recall at k
    def ratk(self, data, k, num_users, num_items, pred, true='rating'):
        """
            data: pandas DataFrame. 
            k: top-k items relevant
            pred: string. Column name that corresponding to the prediction
            true: string. Column name that corresponding to the true rating
        """
        prediction = self.getMatrix(data, num_users, num_items, pred)
        testSet =  self.getMatrix(data, num_users, num_items, true)
        # Initialize sum and count vars for average calculation
        sumRecalls = 0
        countRecalls = 0

        # Define function for converting 1-5 rating to 0/1 (like / don't like)
        vf = np.vectorize(lambda x: 1 if x >= 4 else 0)

        for userID in range(num_users):
            # Pick top K based on predicted rating
            userVector = prediction[userID,:]
            topK = nlargest(k, range(len(userVector)), userVector.take)

            # Convert test set ratings to like / don't like
            userTestVector = vf(testSet[userID,:]).nonzero()[0]

            # Ignore user if has no ratings in the test set
            if (len(userTestVector) == 0):
                continue

            # Calculate recall
            recall = float(len([item for item in topK if item in userTestVector]))/len(userTestVector)

            # Update sum and count
            sumRecalls += recall
            countRecalls += 1

        # Return average R@k
        return float(sumRecalls)/countRecalls
    
    @staticmethod
    def getMatrix(rating_df, num_users, num_items, column_name):
        matrix = np.zeros((num_users, num_items))
    
        for (index, userID, itemID, value) in rating_df[['userID','itemID', column_name]].itertuples():
            matrix[userID-1, itemID-1] = value
            
        return matrix
    
    @staticmethod
    def _getData(data_path):
        """
            Don't change this function
        """
        folds = []
        data_types = ['u{0}.base','u{0}.test']
        for i in range(1,6):
            train_set = getData(data_path, data_types[0].format(i))
            train_set = train_set.loc[train_set['userID'].isin(thresholdbelow)]
            
            test_set = getData(data_path, data_types[1].format(i))
            test_set = test_set.loc[test_set['userID'].isin(thresholdbelow)]
            
            folds.append([train_set, test_set])
        return folds
    
    def run(self, algorithms, num_users, num_items, k=1):
        """
            5-fold cross-validation
            algorithms: list. a list of algorithms. 
                        eg: [user_cosine_recsys, item_euclidean_recsys]
        """
        
        scores = {}
        for algorithm in algorithms:
            print('Processing algorithm {0}'.format(algorithm.getPredColName()))
            fold_scores = []
            for fold in self.folds:
                algorithm.reset()
                algorithm.predict_all(fold[0], num_users, num_items)
                prediction = algorithm.evaluate_test(fold[1])
                pred_col = algorithm.getPredColName()
                fold_scores.append(self.metric(prediction, k, num_users, num_items, pred_col))
                
            mean = np.mean(fold_scores)
            ci_low, ci_high = stats.t.interval(0.95, len(fold_scores)-1, loc=mean, scale=stats.sem(fold_scores))
            scores[algorithm.getPredColName()] = [fold_scores, mean, ci_low, ci_high]
            
        results = scores    
    
        return results

In [74]:
cv_patk = CrossValidationq6below('RMSE')
cv_patk.run(algorithm_instances_Q6, num_users, num_items,k=5)

Processing algorithm user-cosine


14310it [00:04, 3550.91it/s]
14030it [00:04, 3349.37it/s]
14933it [00:04, 3521.63it/s]
15120it [00:04, 3542.19it/s]
15010it [00:04, 3544.16it/s]


Processing algorithm item-cosine


14310it [00:04, 3316.21it/s]
14030it [00:03, 3574.64it/s]
14933it [00:04, 3551.18it/s]
15120it [00:04, 3369.64it/s]
15010it [00:04, 3542.58it/s]


{'user-cosine': [[1.0292964812509109,
   1.0134988342423394,
   1.0162441815962555,
   1.0083129744573012,
   1.0168529851897112],
  1.0168410913473036,
  1.0072353809052537,
  1.0264468017893535],
 'item-cosine': [[1.0462718991797464,
   1.02116589621018,
   1.0146139914163754,
   1.016889536902209,
   1.0147096245053064],
  1.0227301896427634,
  1.0060602497033544,
  1.0394001295821724]}

Q6) Analysis

* For each metric the ave, low cI and high cI are considered.
Below
{'item-cosine': [ 1.022, 1.006, 1.039,
'user-cosine': [ 1.016, 1.007, 1.026}
Above
{'item-cosine': [ 1.051, 1.029, 1.073,
'user-cosine': [ 1.056, 1.042, 1.071]}
Full members 
{'item-cosine': [ 1.020, 1.006, 1.033,
'user-cosine': [ 1.017, 1.009, 1.025,

* The RMSE values of below are less than Above. so the below perfomed better becuase in case for below we have more points (can see the number of iterations) and more vectors so better predictions. In the same way for full members as it has more information than either of the cases it performed slightly better.
* But the above analysis cannot be generalized unless I see the other k values. Even I am changing the K values by 10 there is more diffence. There is tradeoof between number of vectors and number of ratings inside a vector

In [75]:
# Validation
# Constants for validation only
ROW_NUM = 943
COL_NUM = 1682
RATING_COL = 'rating'

In [76]:
def validateDataPreprocessor(path=path, getData=getData, getMatrix=CrossValidation.getMatrix):
    validation_df = getData(path, 'u1.test')
    try:
        matrix = getMatrix(validation_df, ROW_NUM, COL_NUM, RATING_COL)
    except:
        print('dataPreprocessor function has error')
        return
    try:
        assert(matrix.shape == (ROW_NUM,COL_NUM)),\
        "Shape of matrix{0} doesn't match predefined shape (943,1682)".format(matrix.shape)
    except Exception as e:
        print(e)
    return validation_df

In [77]:
validation_df = validateDataPreprocessor()


### Baseline Recommendation Systems

#### Popularity Based Recommendation


In [78]:
def validatePopularityRecSys(validation_df=validation_df, BaseLineRecSys = BaseLineRecSys):
    popularity_recsys = BaseLineRecSys('popularity')
    try:
        popularity_recsys.predict_all(validation_df, ROW_NUM, COL_NUM)
    except Exception as e:        
        print('popularity function has error')
        print(e)
        return
    try:
        predictionMatrix = popularity_recsys.getModel()
        assert(predictionMatrix.shape == (ROW_NUM, COL_NUM)),\
        "Shape of matrix{0} doesn't match predefined shape ({1},{2})"\
        .format(predictionMatrix.shape,ROW_NUM, COL_NUM)
    except Exception as e:
        print(e)

validatePopularityRecSys()

calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


#### User Average Based Recommendation



In [79]:
def validateUserAverRecSys(validation_df=validation_df, BaseLineRecSys = BaseLineRecSys):
    useraverage_recsys = BaseLineRecSys('useraverage')
    try:
        useraverage_recsys.predict_all(validation_df, ROW_NUM, COL_NUM)
    except:
        print('useraverage function has error')
        return
    try:
        predictionMatrix = useraverage_recsys.getModel()
        assert(predictionMatrix.shape == (ROW_NUM, COL_NUM)),\
        "Shape of matrix{0} doesn't match predefined shape ({1},{2})"\
        .format(predictionMatrix.shape,ROW_NUM, COL_NUM)
    except Exception as e:
        print(e)

validateUserAverRecSys()

calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users




### Similary Based Recommendation Systems

#### Euclidean Similarity Function

In [80]:
def validateEuclidean(validation_df=validation_df, getMatrix=CrossValidation.getMatrix):
    matrix = getMatrix(validation_df, ROW_NUM, COL_NUM, RATING_COL)
    try:
        sim_matrix = SimBasedRecSys.euclidean(matrix)
        assert(sim_matrix.shape == (ROW_NUM, ROW_NUM)),\
        "Shape of matrix{0} doesn't match predefined shape ({1},{2})"\
        .format(sim_matrix.shape,ROW_NUM,ROW_NUM)
        assert(np.any(sim_matrix <= 1)),\
               "Exist similarity value that is not less or equal to 1."
    except Exception as e:
        print(e)        

validateEuclidean()

#### Customized Similarity Function (test somethingelse function)

In [81]:
def validateCustomizedSim(validation_df=validation_df, getMatrix=CrossValidation.getMatrix):
    matrix = getMatrix(validation_df, ROW_NUM, COL_NUM, RATING_COL)
    try:
        sim_matrix = SimBasedRecSys.somethingelse(matrix)
        assert(sim_matrix.shape == (ROW_NUM, ROW_NUM)),\
        "Shape of matrix{0} doesn't match predefined shape ({1},{2})"\
        .format(sim_matrix.shape,ROW_NUM,ROW_NUM)
        assert(np.any(sim_matrix <= 1)),\
               "Exist similarity value that is not less or equal to 1."
    except Exception as e:
        print(e) 

validateCustomizedSim()

#### User-User Similarity Based Recommendation System

In [82]:
def validateUUSimBasedRecSys(validation_df=validation_df, dataPreprocessor=dataPreprocessor):
    try:
        user_cosine_recsys = SimBasedRecSys('user','cosine', dataPreprocessor)
    except:
        print("Framework error, please make sure you are using given yml file.")
        return
    
    try:
        user_cosine_recsys.predict_all(validation_df, ROW_NUM, COL_NUM)
        predictionMatrix = user_cosine_recsys.getModel()
        assert(predictionMatrix.shape == (ROW_NUM, COL_NUM)),\
        "Shape of matrix{0} doesn't match predefined shape ({1},{2})"\
        .format(predictionMatrix.shape,ROW_NUM, COL_NUM)
    except Exception as e:
        print(e)

validateUUSimBasedRecSys()

#### Item-Item Similarity Based Recommendation System

In [83]:
def validateIISimBasedRecSys(validation_df=validation_df, dataPreprocessor=dataPreprocessor):
    try:
        user_cosine_recsys = SimBasedRecSys('item','cosine', dataPreprocessor)
    except:
        print("Framework error, please make sure you are using given yml file.")
        return
    
    try:
        user_cosine_recsys.predict_all(validation_df, ROW_NUM, COL_NUM)
        predictionMatrix = user_cosine_recsys.getModel()
        assert(predictionMatrix.shape == (ROW_NUM, COL_NUM)),\
        "Shape of matrix{0} doesn't match predefined shape ({1},{2})"\
        .format(predictionMatrix.shape,ROW_NUM, COL_NUM)
    except Exception as e:
        print(e)

validateIISimBasedRecSys()